In [297]:
import pandas as pd

import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.rcParams['timezone'] = 'Europe/Berlin'

import data_process_base
import importlib
importlib.reload(data_process_base)

try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]

In [298]:
# create random time series from import data
da = data_process_base.DataAction()
keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'cet_cest_timestamp']
da.imp_procc("house_data.csv", keep_cols)
night_merge = da.power_merge()

Number of data frame segments =  90
dfList created successfully.
Whoops, grabbing a different touple of dates...


In [299]:
# # for testing the sgen_comm function
# test = night_merge.iloc[:9,:].copy()
# da.sgen_comm(ts=test, wind_length=2, sgen_val=999999, parties=4)

In [300]:
# assign generator value at random times
sgen_val = 0.01 # typical val: 0.010 MW = 10kW
# night_mw = da.sgen_comm(ts=night_merge, wind_length=60, sgen_val=sgen_val, parties=4)
night_mw = da.sgen_rand(ts=night_merge, sgen_val=sgen_val)


In [301]:
# run timeseries processes
nc = data_process_base.net_calc()

nc.four_loads_branched_make(night_mw)
nc.four_loads_branched_out("res_line", "loading_percent")
nc.four_loads_branched_run()
nc.four_loads_branched_read_loadpct()
# nc.four_loads_branched_plot_linepct()

100%|██████████| 601/601 [00:01<00:00, 339.10it/s]


In [302]:
def end_val_make(night_merge, sgen_val, iter):
    """
    Final boss

    """
    # get inputs from df
    col_names = ['line_1', 'line_2', 'line_3', 'line_4', 'line_5', 'line_6', 'line_7', 'line_8']

    # create df to store output
    end_val = pd.DataFrame(columns=col_names)


    for i in range(0, iter):
        # assign generator value at random times
        # night_mw = da.sgen_comm(ts=night_merge, wind_length=60, sgen_val=sgen_val, parties=4)
        night_mw = da.sgen_rand(ts=night_merge, sgen_val=sgen_val)

        nc.four_loads_branched_make(night_mw)
        nc.four_loads_branched_out("res_line", "loading_percent")
        nc.four_loads_branched_run()
        nc.four_loads_branched_read_loadpct()

        nc.end_val_step(nc.ll, end_val)

    return end_val


iter = 5
sgen_val = 0.01 # typical val: 0.010 MW = 10kW
end_val_make(night_merge, sgen_val, iter)


100%|██████████| 601/601 [00:01<00:00, 357.58it/s]


,line_1,line_2,line_3,line_4,line_5,line_6,line_7,line_8
0,39.540316,33.029353,25.191199,12.568007,19.219220,10.951966,17.854333,12.568195
1,39.540316,33.029353,25.191199,12.568007,19.243038,10.951966,17.512105,12.568195
2,39.324336,32.987826,25.159458,12.568007,19.243038,10.938300,17.854333,12.568195
3,39.540316,33.029353,25.191199,12.568007,19.195671,10.951966,17.831941,12.568195
4,39.540316,33.029353,25.191199,12.568007,19.219220,10.951966,17.854333,12.568195


In [303]:
for i in range(0,5):
    print('bruh')

bruh
bruh
bruh
bruh
bruh


In [304]:
def end_val_make(df):
    """
    append vals to end_val df
    
    """
    # get inputs from df
    col_names = df.columns.tolist()
    max_val = df.max()

    # create df to store output
    end_val = pd.DataFrame(columns=col_names)
    
    # append series as last line
    end_val.loc[end_val.shape[0]] = max_val

    return end_val


end_val_make(nc.ll)

,line_1,line_2,line_3,line_4,line_5,line_6,line_7,line_8
0,39.540316,33.029353,25.191199,12.568007,19.21922,10.951966,17.854333,12.568195


In [305]:
def end_times_make(df):
    """
    append time vals to end_times df
    
    """
    # get inputs
    col_names = df.columns.tolist()
    max_ind = df.idxmax()
    k = nc.night_mw.index.values[max_ind.tolist()]
    max_time = pd.to_datetime(k).strftime('%H:%M:%S').tolist()

    # create df to store output
    end_times = pd.DataFrame(dtype='str', columns=col_names)
    
    # append series as last line
    end_times.loc[end_times.shape[0]] = max_time

    return end_times


# end_times_make(nc.ll)